In [1]:
from revChatGPT.V1 import Chatbot
import json
import time
from tqdm import tqdm

In [2]:
with open('data/karamazov.txt', 'r') as f:
    corpus = f.read()

In [3]:
n = 2500 # chunk length
corpus_chunks = [corpus[i:i+n] for i in range(0, len(corpus), n)]

In [67]:
chatbot = Chatbot({
  "email": "---",
  "password": "---"
}, conversation_id=None, parent_id=None) # 

In [4]:
corpus_chunks[1]

'o or\nthree generations. Adelaïda Ivanovna Miüsov’s action was similarly, no\ndoubt, an echo of other people’s ideas, and was due to the irritation\ncaused by lack of mental freedom. She wanted, perhaps, to show her\nfeminine independence, to override class distinctions and the despotism\nof her family. And a pliable imagination persuaded her, we must\nsuppose, for a brief moment, that Fyodor Pavlovitch, in spite of his\nparasitic position, was one of the bold and ironical spirits of that\nprogressive epoch, though he was, in fact, an ill‐natured buffoon and\nnothing more. What gave the marriage piquancy was that it was preceded\nby an elopement, and this greatly captivated Adelaïda Ivanovna’s fancy.\nFyodor Pavlovitch’s position at the time made him specially eager for\nany such enterprise, for he was passionately anxious to make a career\nin one way or another. To attach himself to a good family and obtain a\ndowry was an alluring prospect. As for mutual love it did not exist\nappar

In [ ]:
for CHUNCK in corpus_chunks:
    prompt =f"""
    The following is part of the Brothers Karamazov novel:

    {CHUNCK}

    Extract only human characters and persons (not places and other entities) from it and understand relationships between each of two characters. make sure to generate the results in the following python dictionary format:

    """ + """{"from": "char1", "to": "char2", "label":"relation"}"""

    response = chatbot.ask(prompt, conversation_id=None, parent_id=None) 

    print(response['message'])
    print("\n-------------------")

    with open('karamazov-graph-only-chars.jsonl', 'a') as f:
        for r in response['message'].split("\n"):
            try:
                json.dump({'chunck':CHUNCK, **json.loads(r)}, f)
                f.write("\n")
            except:
                pass

    time.sleep(300) # to not run out of possible requests in an hour

In [27]:
with open('data/karamazov-graph.json','r') as f:
    data = json.load(f)

In [39]:
for d in tqdm(data['nodes']):
    prompt =f"""
    Write a paragraph about the role of {d['name']} in the Brothers Karamazov Novel and describe the role {d['name']} plays in the novel. 
    In possible cases, add a deeper analyses of different aspects of the character.
    """
    response = chatbot.ask(prompt)
    for r in response:
        message = r["message"]
    with open('data/char-desc.jsonl', 'a') as f:
        json.dump({'id':d['id'],'name':d['name'],'desc':message}, f)
        f.write('\n')
    time.sleep(300)

100%|██████████| 83/83 [7:31:29<00:00, 326.38s/it]  


In [44]:
mapping = {a['id']:a['name'] for a in data['nodes']}

In [68]:
for d in tqdm(data['links']):
    prompt =f"""
    In the borthers karamazov novel {mapping[d['source']]} and {mapping[d['target']]} are interacting with each other in different ways. 
    Describe and analyse the relationship between {mapping[d['source']]} and {mapping[d['target']]}.
    Make sure to only describe the relationship and don't provide any further details on each character."""
    response = chatbot.ask(prompt)
    for r in response:
        message = r["message"]
    with open('data/char-rel.jsonl', 'a') as f:
        json.dump({'source':mapping[d['source']],'target':mapping[d['target']],'desc':message}, f)
        f.write('\n')
    time.sleep(300)

100%|██████████| 42/42 [3:46:48<00:00, 324.01s/it]  
